In [1]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

# Import the requests library. Import the API key
import requests
from config import weather_api_key

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

Here's an outline of your project plan:
- Task: Collect and analyze weather data across cities worldwide.
- Purpose: PlanMyTrip will use the data to recommend ideal hotels based on clients' weather preferences.
- Method: Create a Pandas DataFrame with 500 or more of the world's unique cities and their weather data in real time. This process will entail collecting, analyzing, and visualizing the data.

Your analysis of the data will be split into three main parts, or stages.

1. **Collect the Data**

- Use the NumPy module to generate more than 1,500 random latitudes and longitudes.
- Use the citipy module to list the nearest city to the latitudes and longitudes.
- Use the OpenWeatherMap API to request the current weather data from each unique city in your list.
- Parse the JSON data from the API request.
- Collect the following data from the JSON file and add it to a DataFrame:
    - City, country, and date
    - Latitude and longitude
    - Maximum temperature
    - Humidity
    - Cloudiness
    - Wind speed

2. **Exploratory Analysis with Visualization**

- Create scatter plots of the weather data for the following comparisons:
    - Latitude versus temperature
    - Latitude versus humidity
    - Latitude versus cloudiness
    - Latitude versus wind speed
- Determine the correlations for the following weather data:
    - Latitude and temperature
    - Latitude and humidity
    - Latitude and cloudiness
    - Latitude and wind speed
- Create a series of heatmaps using the Google Maps and Places API that showcases the following:
    - Latitude and temperature
    - Latitude and humidity
    - Latitude and cloudiness
    - Latitude and wind speed
    
3. **Visualize Travel Data**
Create a heatmap with pop-up markers that can display information on specific cities based on a customer's travel preferences. Complete these steps:


1. Filter the Pandas DataFrame based on user inputs for a minimum and maximum temperature.
2. Create a heatmap for the new DataFrame.
3. Find a hotel from the cities' coordinates using Google's Maps and Places API, and Search Nearby feature.
4. Store the name of the first hotel in the DataFrame.
5. Add pop-up markers to the heatmap that display information about the city, current maximum temperature, and a hotel in the city.

# Generate Random Coordinates of World Cities

We use the **geographic coordinate system** (**GCS**) to reference any point on Earth by its latitude and longitude coordinates.

**Latitudes** are imaginary lines on Earth that run parallel east to west and are measured in angular units called degrees, minutes, and seconds, with 60 minutes in a degree and 60 seconds in a minute. Sometimes a latitude is referred to as a **parallel**.

The **equator** is an imaginary line around the middle of the earth that is equidistant from the North and South Poles and has a latitude of 0°. The equator splits Earth into Northern and Southern Hemispheres.

> All latitude lines above the equator are measured northward and considered positive, after 0° (the equator) and up to 90°, or 90° north (the North Pole). All latitude lines below the equator are measured southward and considered negative, before 0° (the equator) and down to -90°, or 90° south (the South Pole).

**Longitudes** are imaginary lines on Earth that run from the North to the South Poles and are called **meridians**. The prime meridian represents zero meridian, the origin for longitude coordinates, and splits Earth into the Eastern and Western Hemispheres.

> All meridians east of the prime meridian are considered positive, after 0° and up to 180°. All meridians west of the prime meridian are considered negative, before 0° and down to –180°.

## Generate Random Latitudes and Longitudes

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

78.12784105785033 -4.085199424548421
47.97568014685797 2.3220730271741274
-51.6121396907438 -151.02222206707634
41.63423455939389 67.71487586457926
21.2587017828329 -71.47048229085632
48.0313393607872 -111.05117440497932
-59.04336806871989 -21.373170579812893
-52.51162154395567 87.58882068980978
-29.253030709463168 -0.9566651408473774
65.52496305644695 20.000249507997637
-35.078319703433756 -53.17227593466134
21.913870044365666 -57.923897967875945
-87.53380716727769 89.7077103105226
31.207700187380325 -172.25134979793853
-40.71812630928911 171.27285923727186
11.179606788823065 161.53944286015286
43.908502392694544 -106.16916190843591
-8.379763733670245 -132.98785865646363
88.25912615366519 14.844226249247413
83.93432567606399 5.599841994849754
-9.819065092631647 70.89533913175606
-72.18070875829636 -97.2943500290933
-52.20560440339537 164.0328613818238
75.16675708719686 -89.17362062227286
-11.94387796079323 -121.2996951543857
50.5956869858739 -70.51429337982216
-58.489329744356056 -157

## Generate Random World Cities

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

617

# Retrieve, Collect and Clean Weather Data

## Get the City Weather Data

For each city in our **lats_lngs** list, we need to retrieve the following data and add it to a DataFrame:
- City, country, and date
- Latitude and longitude
- Maximum temperature
- Humidity
- Cloudiness
- Wind speed

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=metric&APPID=" + weather_api_key

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | klaksvik
Processing Record 2 of Set 1 | pithiviers
Processing Record 3 of Set 1 | mataura
Processing Record 4 of Set 1 | chardara
Processing Record 5 of Set 1 | cockburn harbour
City not found. Skipping...
Processing Record 6 of Set 1 | great falls
Processing Record 7 of Set 1 | cidreira
Processing Record 8 of Set 1 | busselton
Processing Record 9 of Set 1 | jamestown
Processing Record 10 of Set 1 | skelleftea
Processing Record 11 of Set 1 | rocha
Processing Record 12 of Set 1 | codrington
Processing Record 13 of Set 1 | albany
Processing Record 14 of Set 1 | kapaa
Processing Record 15 of Set 1 | karamea
City not found. Skipping...
Processing Record 16 of Set 1 | butaritari
Processing Record 17 of Set 1 | gillette
Processing Record 18 of Set 1 | atuona
Processing Record 19 of Set 1 | longyearbyen
Processing Record 20 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 21 of Set 1

Processing Record 35 of Set 4 | pontes e lacerda
Processing Record 36 of Set 4 | yumen
Processing Record 37 of Set 4 | isangel
Processing Record 38 of Set 4 | varnamo
Processing Record 39 of Set 4 | ancud
Processing Record 40 of Set 4 | baiquan
Processing Record 41 of Set 4 | gawler
Processing Record 42 of Set 4 | omsukchan
Processing Record 43 of Set 4 | hermanus
Processing Record 44 of Set 4 | hasaki
Processing Record 45 of Set 4 | longview
Processing Record 46 of Set 4 | cabo san lucas
Processing Record 47 of Set 4 | jijiga
Processing Record 48 of Set 4 | victoria
Processing Record 49 of Set 4 | kahului
Processing Record 50 of Set 4 | tautira
Processing Record 1 of Set 5 | sakakah
Processing Record 2 of Set 5 | long xuyen
Processing Record 3 of Set 5 | champerico
Processing Record 4 of Set 5 | nyurba
Processing Record 5 of Set 5 | schneeberg
Processing Record 6 of Set 5 | alice springs
Processing Record 7 of Set 5 | soyo
Processing Record 8 of Set 5 | aswan
Processing Record 9 of Se

Processing Record 23 of Set 8 | mafeteng
Processing Record 24 of Set 8 | fairview
Processing Record 25 of Set 8 | juneau
Processing Record 26 of Set 8 | san vicente
Processing Record 27 of Set 8 | karamadai
Processing Record 28 of Set 8 | gogrial
Processing Record 29 of Set 8 | kayerkan
Processing Record 30 of Set 8 | chase
Processing Record 31 of Set 8 | paucartambo
City not found. Skipping...
Processing Record 32 of Set 8 | prince rupert
Processing Record 33 of Set 8 | palmer
Processing Record 34 of Set 8 | tambul
City not found. Skipping...
Processing Record 35 of Set 8 | makat
Processing Record 36 of Set 8 | artyk
City not found. Skipping...
Processing Record 37 of Set 8 | taoudenni
Processing Record 38 of Set 8 | codajas
Processing Record 39 of Set 8 | nome
Processing Record 40 of Set 8 | saint anthony
Processing Record 41 of Set 8 | cayenne
Processing Record 42 of Set 8 | laguna
Processing Record 43 of Set 8 | hachinohe
Processing Record 44 of Set 8 | drumheller
Processing Record

Processing Record 7 of Set 12 | bundi
Processing Record 8 of Set 12 | puerto madero
Processing Record 9 of Set 12 | san ramon
Processing Record 10 of Set 12 | willowmore
Processing Record 11 of Set 12 | langsa
Processing Record 12 of Set 12 | neryungri
Processing Record 13 of Set 12 | poltavka
Processing Record 14 of Set 12 | fenoarivo
Processing Record 15 of Set 12 | kilindoni
Processing Record 16 of Set 12 | broome
Processing Record 17 of Set 12 | cabra
Processing Record 18 of Set 12 | nishihara
Processing Record 19 of Set 12 | college
Processing Record 20 of Set 12 | saint-joseph
Processing Record 21 of Set 12 | mahajanga
Processing Record 22 of Set 12 | khorixas
Processing Record 23 of Set 12 | ganzhou
Processing Record 24 of Set 12 | mugumu
Processing Record 25 of Set 12 | filingue
Processing Record 26 of Set 12 | rungata
City not found. Skipping...
Processing Record 27 of Set 12 | roma
Processing Record 28 of Set 12 | padang
Processing Record 29 of Set 12 | jamui
Processing Recor

In [10]:
city_data

[{'City': 'Klaksvik',
  'Lat': 62.2266,
  'Lng': -6.589,
  'Max Temp': 11.51,
  'Humidity': 93,
  'Cloudiness': 75,
  'Wind Speed': 7.09,
  'Country': 'FO',
  'Date': '2022-08-09 01:59:21'},
 {'City': 'Pithiviers',
  'Lat': 48.1667,
  'Lng': 2.25,
  'Max Temp': 19.9,
  'Humidity': 57,
  'Cloudiness': 0,
  'Wind Speed': 5.22,
  'Country': 'FR',
  'Date': '2022-08-09 01:59:22'},
 {'City': 'Mataura',
  'Lat': -46.1927,
  'Lng': 168.8643,
  'Max Temp': 7.24,
  'Humidity': 57,
  'Cloudiness': 5,
  'Wind Speed': 3.45,
  'Country': 'NZ',
  'Date': '2022-08-09 01:59:22'},
 {'City': 'Chardara',
  'Lat': 41.2547,
  'Lng': 67.9692,
  'Max Temp': 24.69,
  'Humidity': 45,
  'Cloudiness': 1,
  'Wind Speed': 2.26,
  'Country': 'KZ',
  'Date': '2022-08-09 01:59:23'},
 {'City': 'Great Falls',
  'Lat': 47.5002,
  'Lng': -111.3008,
  'Max Temp': 33.66,
  'Humidity': 10,
  'Cloudiness': 0,
  'Wind Speed': 0.89,
  'Country': 'US',
  'Date': '2022-08-09 01:58:03'},
 {'City': 'Cidreira',
  'Lat': -30.1811,
 

## Create a DataFrame of City Weather Data

In [12]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Klaksvik,62.2266,-6.5890,11.51,93,75,7.09,FO,2022-08-09 01:59:21
1,Pithiviers,48.1667,2.2500,19.90,57,0,5.22,FR,2022-08-09 01:59:22
2,Mataura,-46.1927,168.8643,7.24,57,5,3.45,NZ,2022-08-09 01:59:22
3,Chardara,41.2547,67.9692,24.69,45,1,2.26,KZ,2022-08-09 01:59:23
4,Great Falls,47.5002,-111.3008,33.66,10,0,0.89,US,2022-08-09 01:58:03
...,...,...,...,...,...,...,...,...,...
553,Vila Velha,-20.3297,-40.2925,23.97,87,0,3.09,BR,2022-08-09 02:16:12
554,Merke,42.8728,73.1786,23.99,44,3,0.24,KZ,2022-08-09 02:16:13
555,Ust-Nera,64.5667,143.2000,23.64,42,4,1.05,RU,2022-08-09 02:16:13
556,Lakhdenpokhya,61.5198,30.1992,13.85,70,99,1.29,RU,2022-08-09 02:16:14


In [14]:
city_data_columnorder = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 
                         'Cloudiness', 'Wind Speed']
city_data_ardf = city_data_df [city_data_columnorder]
city_data_ardf

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Klaksvik,FO,2022-08-09 01:59:21,62.2266,-6.5890,11.51,93,75,7.09
1,Pithiviers,FR,2022-08-09 01:59:22,48.1667,2.2500,19.90,57,0,5.22
2,Mataura,NZ,2022-08-09 01:59:22,-46.1927,168.8643,7.24,57,5,3.45
3,Chardara,KZ,2022-08-09 01:59:23,41.2547,67.9692,24.69,45,1,2.26
4,Great Falls,US,2022-08-09 01:58:03,47.5002,-111.3008,33.66,10,0,0.89
...,...,...,...,...,...,...,...,...,...
553,Vila Velha,BR,2022-08-09 02:16:12,-20.3297,-40.2925,23.97,87,0,3.09
554,Merke,KZ,2022-08-09 02:16:13,42.8728,73.1786,23.99,44,3,0.24
555,Ust-Nera,RU,2022-08-09 02:16:13,64.5667,143.2000,23.64,42,4,1.05
556,Lakhdenpokhya,RU,2022-08-09 02:16:14,61.5198,30.1992,13.85,70,99,1.29


In [15]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")